In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
import cPickle
import os
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

In [2]:
def getFrameData(root, iterKey):
    allFrameData = []
    for frameData in root.iter(iterKey):
         allFrameData.append(frameData.attrib)
    return allFrameData

def readXMLFile(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot()
    parent = {}
    for child in root:
        tempList = list(child)
    #     print child.tag
        if not tempList:
            if not child.attrib:
                parent[child.tag] = child.text
            else:
                parent[child.tag] = child.attrib
        else:
            parent[child.tag] = {}
            for grand_child in child:
    #             print '\t', grand_child.tag
                tempList = list(grand_child)
                if not tempList:
                    if not grand_child.attrib:
                        parent[child.tag][grand_child.tag] = grand_child.text
                    else:
                        parent[child.tag][grand_child.tag] = grand_child.attrib
                else:
                    parent[child.tag][grand_child.tag] = {}
                    for grand_grand_child in grand_child:
    #                     print '\t\t', grand_grand_child.tag
                        tempList = list(grand_grand_child)
                        if not tempList:
                            if not grand_grand_child.attrib:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.text
                            else:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.attrib
                        else:
                            parent[child.tag][grand_child.tag][grand_grand_child.tag] = {}
                            for grand_grand_grand_child in grand_grand_child:
    #                             print '\t\t\t', grand_grand_grand_child.tag
                                tempList = list(grand_grand_grand_child)
                                if not tempList:
                                    if not grand_grand_grand_child.attrib:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                    else:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                else:
                                    parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = {}
                                    for grand_grand_grand_grand_child in grand_grand_grand_child:
            #                             print '\t\t\t', grand_grand_grand_grand_child.tag
                                        tempList = list(grand_grand_grand_grand_child)
                                        if not tempList:
                                            if not grand_grand_grand_grand_child.attrib:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                            else:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                        else:
                                            print "There are more grand childrens than you think"

    parent['kinetics'] = []
    parent['position'] = {}
    parent['ruleData'] = []
    parent['stimulus'] = []

    larvaFrameData = pd.DataFrame(getFrameData(root, 'larvaFrameData'))
    skeleton = pd.DataFrame(getFrameData(root, 'skeleton'))
    parent['kinetics'] = pd.concat([larvaFrameData, skeleton])
    parent['position']['midpoint'] = pd.DataFrame(getFrameData(root, 'midpoint'))
    parent['position']['centroid'] = pd.DataFrame(getFrameData(root, 'centroid'))
    parent['position']['head'] = pd.DataFrame(getFrameData(root, 'head'))
    parent['position']['tail'] = pd.DataFrame(getFrameData(root, 'tail'))
    parent['position']['featureLocation'] = pd.DataFrame(getFrameData(root, 'featureLocation'))
    parent['ruleData'] = pd.DataFrame(getFrameData(root, 'ruleData'))

    stimulusIntensity = pd.DataFrame(getFrameData(root, 'intensityPercentage'))
    temp = pd.DataFrame(getFrameData(root, 'ledArrayStimulus'))
    if not temp.empty:
        parent['stimulus'] = pd.concat([temp, stimulusIntensity])
    else:
        parent['stimulus'] = stimulusIntensity

    return parent

In [3]:
# file_name = 'sample_landscape.xml'
# file_name = 'sample_new_tracker.xml'
# file_name = 'sample_rule_data.xml'
# all_xml_files = [file_name, file_name]

all_xml_files = []
# data_dir = '.'
data_dir = '/Volumes/GoogleDrive/My Drive/CRG_Dropbox/AljoComputer/Matlab/high_res_tracker/Data/20180316_OK6_CsChrimson_Tail_200micron/XML'
for file in os.listdir(data_dir):
    if file.endswith(".xml"):
        all_xml_files.append(os.path.join(data_dir, file))

In [4]:
## Parse Files in Parallel
tStart = time.time()
kin_data = []
kin_data = Parallel(n_jobs=num_cores)(delayed(readXMLFile)(xml_file) for xml_file in all_xml_files)
print 'Time taken to parse %d files in parallel is %0.3f sec'%(len(all_xml_files), time.time()-tStart)

Time taken to parse 15 files in parallel is 284.243 sec


In [5]:
## Save Parsed Files
with open(os.path.join(data_dir, 'kinData_xml.pkl'), 'wb') as write_file:
    cPickle.dump(kin_data, write_file, protocol=2)

In [6]:
## Parse Files Sequentially
# c=0
# for xml_file in all_xml_files:
#     tStart = time.time()
#     temp = readXMLFile(xml_file)
#     kin_data.append(readXMLFile(xml_file))
#     c+=1
#     print 'Time taken to parse file %d is %0.3f'%(c, time.time()-tStart)

In [5]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
import cPickle
import os

def getFrameData(root, iterKey):
    allFrameData = []
    for frameData in root.iter(iterKey):
         allFrameData.append(frameData.attrib)
    return allFrameData

file_name = 'sample_new_tracker.xml'
tree = ET.parse(file_name)
root = tree.getroot()
parent = {}
for child in root:
    tempList = list(child)
#     print child.tag
    if not tempList:
        if not child.attrib:
            parent[child.tag] = child.text
        else:
            parent[child.tag] = child.attrib
    else:
        parent[child.tag] = {}
        for grand_child in child:
#             print '\t', grand_child.tag
            tempList = list(grand_child)
            if not tempList:
                if not grand_child.attrib:
                    parent[child.tag][grand_child.tag] = grand_child.text
                else:
                    parent[child.tag][grand_child.tag] = grand_child.attrib
            else:
                parent[child.tag][grand_child.tag] = {}
                for grand_grand_child in grand_child:
#                     print '\t\t', grand_grand_child.tag
                    tempList = list(grand_grand_child)
                    if not tempList:
                        if not grand_grand_child.attrib:
                            parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.text
                        else:
                            parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.attrib
                    else:
                        parent[child.tag][grand_child.tag][grand_grand_child.tag] = {}
                        for grand_grand_grand_child in grand_grand_child:
#                             print '\t\t\t', grand_grand_grand_child.tag
                            tempList = list(grand_grand_grand_child)
                            if not tempList:
                                if not grand_grand_grand_child.attrib:
                                    parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                else:
                                    parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                            else:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = {}
                                for grand_grand_grand_grand_child in grand_grand_grand_child:
        #                             print '\t\t\t', grand_grand_grand_grand_child.tag
                                    tempList = list(grand_grand_grand_grand_child)
                                    if not tempList:
                                        if not grand_grand_grand_grand_child.attrib:
                                            parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                        else:
                                            parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                    else:
                                        print "There are more grand childrens than you think"

parent['kinetics'] = []
parent['position'] = {}
parent['ruleData'] = []
parent['stimulus'] = []

larvaFrameData = pd.DataFrame(getFrameData(root, 'larvaFrameData'))
skeleton = pd.DataFrame(getFrameData(root, 'skeleton'))
parent['kinetics'] = pd.concat([larvaFrameData, skeleton])
parent['position']['midpoint'] = pd.DataFrame(getFrameData(root, 'midpoint'))
parent['position']['centroid'] = pd.DataFrame(getFrameData(root, 'centroid'))
parent['position']['head'] = pd.DataFrame(getFrameData(root, 'head'))
parent['position']['tail'] = pd.DataFrame(getFrameData(root, 'tail'))
parent['position']['featureLocation'] = pd.DataFrame(getFrameData(root, 'featureLocation'))
parent['ruleData'] = pd.DataFrame(getFrameData(root, 'ruleData'))

stimulusIntensity = pd.DataFrame(getFrameData(root, 'intensityPercentage'))
print stimulusIntensity
temp = pd.DataFrame(getFrameData(root, 'ledArrayStimulus'))
if not temp.empty:
    parent['stimulus'] = pd.concat([temp, stimulusIntensity])
else:
    parent['stimulus'] = stimulusIntensity

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[33008 rows x 0 columns]


In [13]:
for frameData in root.iter('intensityPercentage'):
    print frameData.text

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
